##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Regularização de grafos para classificação de documentos utilizando grafos naturais

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_mlp_cora"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/neural_structured_learning/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/neural_structured_learning/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/neural_structured_learning/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

A regularização de grafos é uma técnica específica sob o paradigma mais amplo de Aprendizagem de Grafos Neurais ("Neural Graph Learning", [Bui et al., 2018](https://research.google/pubs/pub46568.pdf)). A ideia central é treinar modelos de redes neurais com um objetivo regularizado por grafos, aproveitando dados rotulados e não rotulados.

Neste tutorial, exploraremos o uso da regularização de grafos para classificar documentos que formam um grafo natural (orgânico).

A receita geral para criar um modelo regularizado por grafos usando o framework Neural Structured Learning (NSL) é a seguinte:

1. Gere dados de treinamento a partir do grafo de entrada e de características de amostra. Os nós do grafo correspondem a amostras e as arestas do grafo correspondem à similaridade entre pares de amostras. Os dados de treinamento resultantes conterão características vizinhas além das características originais do nó.
2. Crie uma rede neural como modelo de referência usando a API sequencial, funcional ou de subclasse do `Keras`.
3. Envolva o modelo de referência com a classe wrapper **`GraphRegularization`**, que é fornecida pelo framework NSL, para criar um novo modelo de grafo `Keras`. Este novo modelo incluirá uma perda de regularização do grafo como termo de regularização em seu objetivo de treinamento.
4. Treine e avalie o modelo do grafo `Keras`.

## Configuração


Instale o pacote Neural Structured Learning.

In [ ]:
!pip install --quiet neural-structured-learning

## Dependências e importações

In [ ]:
import neural_structured_learning as nsl

import tensorflow as tf

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print(
    "GPU is",
    "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

## Dataset Cora

O [dataset Cora](https://linqs.soe.ucsc.edu/data) é um grafo de citações onde os nós representam artigos de aprendizado de máquina e as bordas representam citações entre pares de artigos. A tarefa envolvida é a classificação de documentos onde o objetivo é categorizar cada artigo em uma das 7 categorias. Em outras palavras, este é um problema de classificação multiclasse com 7 classes.

### Grafo

O grafo original é direcionado. No entanto, para efeitos deste exemplo, consideraremos a versão não direcionada deste grafo. Portanto, se o artigo A cita o artigo B, também consideramos que o artigo B citou A. Embora isto não seja necessariamente verdade, neste exemplo, consideramos as citações como substituto para similaridade, que geralmente é uma propriedade comutativa.

### Características

Cada artigo na entrada contém efetivamente 2 características:

1. **Words**: uma representação densa e multi-hot do texto no artigo. O vocabulário do dataset Cora contém 1.433 palavras únicas. Portanto, o comprimento dessa característica é 1433, e o valor na posição 'i' é 0/1, indicando se a palavra 'i' no vocabulário existe ou não no artigo em questão.

2. **Label**: Um único inteiro representando o ID da classe (categoria) do artigo.

### Baixe o dataset Cora

In [ ]:
!wget --quiet -P /tmp https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
!tar -C /tmp -xvzf /tmp/cora.tgz

### Converta os dados do Cora para o formato NSL

Para pré-processar o dataset Cora e convertê-lo para o formato exigido pelo Neural Structured Learning, executaremos o script **'preprocess_cora_dataset.py'**, que está incluído no repositório NSL github. Este script faz o seguinte:

1. Gere características vizinhas usando as características do nó original e o grafo.
2. Gere divisões de dados de treinamento e teste contendo instâncias `tf.train.Example`.
3. Persista os dados de treinamento e teste resultantes no formato `TFRecord`.

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/neural-structured-learning/master/neural_structured_learning/examples/preprocess/cora/preprocess_cora_dataset.py

!python preprocess_cora_dataset.py \
--input_cora_content=/tmp/cora/cora.content \
--input_cora_graph=/tmp/cora/cora.cites \
--max_nbrs=5 \
--output_train_data=/tmp/cora/train_merged_examples.tfr \
--output_test_data=/tmp/cora/test_examples.tfr

## Variáveis ​​globais

Os caminhos de arquivo para os dados de treinamento e teste são baseados nos valores dos flags de linha de comando usados ​​para executar o script **'preprocess_cora_dataset.py'** acima.

In [ ]:
### Experiment dataset
TRAIN_DATA_PATH = '/tmp/cora/train_merged_examples.tfr'
TEST_DATA_PATH = '/tmp/cora/test_examples.tfr'

### Constants used to identify neighbor features in the input.
NBR_FEATURE_PREFIX = 'NL_nbr_'
NBR_WEIGHT_SUFFIX = '_weight'

## Hiperparâmetros

Usaremos uma instância de `HParams` para incluir vários hiperparâmetros e constantes usados ​​para treinamento e avaliação. Descrevemos brevemente cada um deles abaixo:

- **num_classes**: há um total de 7 classes diferentes

- **max_seq_length**: este é o tamanho do vocabulário e todas as instâncias na entrada têm uma representação bag-of-words (saco de palavras) densa e multi-hot. Em outras palavras, um valor 1 para uma palavra indica que a palavra está presente na entrada e um valor 0 indica que não está.

- **distance_type**: é a métrica de distância usada para regularizar a amostra com seus vizinhos.

- **graph_regularization_multiplier**: controla o peso relativo do termo de regularização do grafo na função de perda geral.

- **num_neighbors**: o número de vizinhos usados ​​para regularização do grafo. Este valor deve ser menor ou igual ao argumento da linha de comando `max_nbrs` usado acima ao executar `preprocess_cora_dataset.py`.

- **num_fc_units**: o número de camadas totalmente conectadas em nossa rede neural.

- **train_epochs**: o número de épocas de treinamento.

- **batch_size**: tamanho do lote usado para treinamento e avaliação.

- **dropout_rate**: controla a taxa de dropout após cada camada totalmente conectada

- **eval_steps**: o número de lotes a serem processados ​​antes que a avaliação seja concluída. Se definido como `None`, todas as instâncias no dataset de testes serão avaliadas.

In [ ]:
class HParams(object):
  """Hyperparameters used for training."""
  def __init__(self):
    ### dataset parameters
    self.num_classes = 7
    self.max_seq_length = 1433
    ### neural graph learning parameters
    self.distance_type = nsl.configs.DistanceType.L2
    self.graph_regularization_multiplier = 0.1
    self.num_neighbors = 1
    ### model architecture
    self.num_fc_units = [50, 50]
    ### training parameters
    self.train_epochs = 100
    self.batch_size = 128
    self.dropout_rate = 0.5
    ### eval parameters
    self.eval_steps = None  # All instances in the test set are evaluated.

HPARAMS = HParams()

## Carregue os dados de treinamento e teste

Conforme descrito anteriormente neste notebook, os dados de treinamento e teste de entrada foram criados pelo **'preprocess_cora_dataset.py'**. Iremos carregá-los em dois objetos `tf.data.Dataset` - um para treinamento e outro para teste.

Na camada de entrada do nosso modelo, extrairemos não apenas as características 'words' e 'label' de cada amostra, mas também as características vizinhas correspondentes com base no valor `hparams.num_neighbors`. Instâncias com menos vizinhos que `hparams.num_neighbors` receberão valores fictícios para essas características vizinhas inexistentes.

In [ ]:
def make_dataset(file_path, training=False):
  """Creates a `tf.data.TFRecordDataset`.

  Args:
    file_path: Name of the file in the `.tfrecord` format containing
      `tf.train.Example` objects.
    training: Boolean indicating if we are in training mode.

  Returns:
    An instance of `tf.data.TFRecordDataset` containing the `tf.train.Example`
    objects.
  """

  def parse_example(example_proto):
    """Extracts relevant fields from the `example_proto`.

    Args:
      example_proto: An instance of `tf.train.Example`.

    Returns:
      A pair whose first value is a dictionary containing relevant features
      and whose second value contains the ground truth label.
    """
    # The 'words' feature is a multi-hot, bag-of-words representation of the
    # original raw text. A default value is required for examples that don't
    # have the feature.
    feature_spec = {
        'words':
            tf.io.FixedLenFeature([HPARAMS.max_seq_length],
                                  tf.int64,
                                  default_value=tf.constant(
                                      0,
                                      dtype=tf.int64,
                                      shape=[HPARAMS.max_seq_length])),
        'label':
            tf.io.FixedLenFeature((), tf.int64, default_value=-1),
    }
    # We also extract corresponding neighbor features in a similar manner to
    # the features above during training.
    if training:
      for i in range(HPARAMS.num_neighbors):
        nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
        nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, i,
                                         NBR_WEIGHT_SUFFIX)
        feature_spec[nbr_feature_key] = tf.io.FixedLenFeature(
            [HPARAMS.max_seq_length],
            tf.int64,
            default_value=tf.constant(
                0, dtype=tf.int64, shape=[HPARAMS.max_seq_length]))

        # We assign a default value of 0.0 for the neighbor weight so that
        # graph regularization is done on samples based on their exact number
        # of neighbors. In other words, non-existent neighbors are discounted.
        feature_spec[nbr_weight_key] = tf.io.FixedLenFeature(
            [1], tf.float32, default_value=tf.constant([0.0]))

    features = tf.io.parse_single_example(example_proto, feature_spec)

    label = features.pop('label')
    return features, label

  dataset = tf.data.TFRecordDataset([file_path])
  if training:
    dataset = dataset.shuffle(10000)
  dataset = dataset.map(parse_example)
  dataset = dataset.batch(HPARAMS.batch_size)
  return dataset


train_dataset = make_dataset(TRAIN_DATA_PATH, training=True)
test_dataset = make_dataset(TEST_DATA_PATH)

Vamos dar uma olhada no dataset de treinamento para ver seu conteúdo.

In [ ]:
for feature_batch, label_batch in train_dataset.take(1):
  print('Feature list:', list(feature_batch.keys()))
  print('Batch of inputs:', feature_batch['words'])
  nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, 0, 'words')
  nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, 0, NBR_WEIGHT_SUFFIX)
  print('Batch of neighbor inputs:', feature_batch[nbr_feature_key])
  print('Batch of neighbor weights:',
        tf.reshape(feature_batch[nbr_weight_key], [-1]))
  print('Batch of labels:', label_batch)

Vamos dar uma olhada no dataset de teste para ver seu conteúdo.

In [ ]:
for feature_batch, label_batch in test_dataset.take(1):
  print('Feature list:', list(feature_batch.keys()))
  print('Batch of inputs:', feature_batch['words'])
  print('Batch of labels:', label_batch)

## Definição do modelo

Para demonstrar o uso da regularização de grafos, construímos primeiro um modelo de referência para este problema. Usaremos uma rede neural feed-forward simples com 2 camadas ocultas e dropout entre elas. Ilustramos a criação do modelo de referência usando todos os tipos de modelo suportados pelo framework `tf.Keras`: sequencial, funcional e subclasse.

### Modelo de referência sequencial

In [ ]:
def make_mlp_sequential_model(hparams):
  """Creates a sequential multi-layer perceptron model."""
  model = tf.keras.Sequential()
  model.add(
      tf.keras.layers.InputLayer(
          input_shape=(hparams.max_seq_length,), name='words'))
  # Input is already one-hot encoded in the integer format. We cast it to
  # floating point format here.
  model.add(
      tf.keras.layers.Lambda(lambda x: tf.keras.backend.cast(x, tf.float32)))
  for num_units in hparams.num_fc_units:
    model.add(tf.keras.layers.Dense(num_units, activation='relu'))
    # For sequential models, by default, Keras ensures that the 'dropout' layer
    # is invoked only during training.
    model.add(tf.keras.layers.Dropout(hparams.dropout_rate))
  model.add(tf.keras.layers.Dense(hparams.num_classes))
  return model

### Modelo de referência funcional

In [ ]:
def make_mlp_functional_model(hparams):
  """Creates a functional API-based multi-layer perceptron model."""
  inputs = tf.keras.Input(
      shape=(hparams.max_seq_length,), dtype='int64', name='words')

  # Input is already one-hot encoded in the integer format. We cast it to
  # floating point format here.
  cur_layer = tf.keras.layers.Lambda(
      lambda x: tf.keras.backend.cast(x, tf.float32))(
          inputs)

  for num_units in hparams.num_fc_units:
    cur_layer = tf.keras.layers.Dense(num_units, activation='relu')(cur_layer)
    # For functional models, by default, Keras ensures that the 'dropout' layer
    # is invoked only during training.
    cur_layer = tf.keras.layers.Dropout(hparams.dropout_rate)(cur_layer)

  outputs = tf.keras.layers.Dense(hparams.num_classes)(cur_layer)

  model = tf.keras.Model(inputs, outputs=outputs)
  return model

### Modelo de referência de subclasse

In [ ]:
def make_mlp_subclass_model(hparams):
  """Creates a multi-layer perceptron subclass model in Keras."""

  class MLP(tf.keras.Model):
    """Subclass model defining a multi-layer perceptron."""

    def __init__(self):
      super(MLP, self).__init__()
      # Input is already one-hot encoded in the integer format. We create a
      # layer to cast it to floating point format here.
      self.cast_to_float_layer = tf.keras.layers.Lambda(
          lambda x: tf.keras.backend.cast(x, tf.float32))
      self.dense_layers = [
          tf.keras.layers.Dense(num_units, activation='relu')
          for num_units in hparams.num_fc_units
      ]
      self.dropout_layer = tf.keras.layers.Dropout(hparams.dropout_rate)
      self.output_layer = tf.keras.layers.Dense(hparams.num_classes)

    def call(self, inputs, training=False):
      cur_layer = self.cast_to_float_layer(inputs['words'])
      for dense_layer in self.dense_layers:
        cur_layer = dense_layer(cur_layer)
        cur_layer = self.dropout_layer(cur_layer, training=training)

      outputs = self.output_layer(cur_layer)

      return outputs

  return MLP()

## Crie modelos base

In [ ]:
# Create a base MLP model using the functional API.
# Alternatively, you can also create a sequential or subclass base model using
# the make_mlp_sequential_model() or make_mlp_subclass_model() functions
# respectively, defined above. Note that if a subclass model is used, its
# summary cannot be generated until it is built.
base_model_tag, base_model = 'FUNCTIONAL', make_mlp_functional_model(HPARAMS)
base_model.summary()

## Treine o modelo de referência MLP

In [ ]:
# Compile and train the base MLP model
base_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
base_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

## Avalie o modelo de referência MLP

In [ ]:
# Helper function to print evaluation metrics.
def print_metrics(model_desc, eval_metrics):
  """Prints evaluation metrics.

  Args:
    model_desc: A description of the model.
    eval_metrics: A dictionary mapping metric names to corresponding values. It
      must contain the loss and accuracy metrics.
  """
  print('\n')
  print('Eval accuracy for ', model_desc, ': ', eval_metrics['accuracy'])
  print('Eval loss for ', model_desc, ': ', eval_metrics['loss'])
  if 'graph_loss' in eval_metrics:
    print('Eval graph loss for ', model_desc, ': ', eval_metrics['graph_loss'])

In [ ]:
eval_results = dict(
    zip(base_model.metrics_names,
        base_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('Base MLP model', eval_results)

## Treine o modelo de referência MLP com regularização de grafo

Incorporar a regularização de grafos no termo de perdas de um `tf.Keras.Model` existente requer apenas umas poucas linhas de código. O modelo de referência é empacotado para criar um novo modelo de subclasse `tf.Keras`, cuja perda inclui a regularização do grafo.

Para avaliar o benefício incremental da regularização de grafos, criaremos uma nova instância do modelo de referência. Isto é necessário porque `base_model` já foi treinado por algumas iterações, e reutilizar esse modelo treinado para criar um modelo regularizado por grafo não produziria uma comparação justa para `base_model`.

In [ ]:
# Build a new base MLP model.
base_reg_model_tag, base_reg_model = 'FUNCTIONAL', make_mlp_functional_model(
    HPARAMS)

In [ ]:
# Wrap the base MLP model with graph regularization.
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=HPARAMS.num_neighbors,
    multiplier=HPARAMS.graph_regularization_multiplier,
    distance_type=HPARAMS.distance_type,
    sum_over_axis=-1)
graph_reg_model = nsl.keras.GraphRegularization(base_reg_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
graph_reg_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

## Avalie o modelo de referência MLP com regularização de grafo

In [ ]:
eval_results = dict(
    zip(graph_reg_model.metrics_names,
        graph_reg_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('MLP + graph regularization', eval_results)

A exatidão do modelo com regularização de grafos é cerca de 2-3% maior que a do modelo de referência (`base_model`).

## Conclusão

Demonstramos o uso da regularização de grafos para classificação de documentos em um grafo de citações naturais (Cora) usando o framework Neural Structured Learning (NSL). Nosso [tutorial avançado](graph_keras_lstm_imdb.ipynb) envolve a síntese de grafos com base em exemplos de embeddings antes de treinar uma rede neural com regularização de grafos. Esta abordagem é útil se a entrada não contiver um grafo explícito.

Incentivamos os usuários a experimentar mais, variando a quantidade de supervisão, bem como experimentando diferentes arquiteturas neurais para a regularização de grafos.